In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras

In [11]:
a=pd.read_csv("C:/Users/srbeh/Desktop/desktop docs/Final Year Project/data from simulation/model18/export.csv",low_memory=False)

In [12]:
df1=a
df=df1/10000
df

,X [ m ],Y [ m ],Z [ m ],Approximated Mass Flow [ kg s^-1 ],Area [ m^2 ],Connectivity Number,Density [ kg m^-3 ],Eddy Viscosity [ Pa s ],Edge Length Ratio,Element Volume Ratio,...,Velocity.Swirling Normal Z [ s^-1 ],Velocity.Swirling Strength [ s^-1 ],Velocity.Swirling Vector [ s^-1 ],Velocity.Swirling Vector X [ s^-1 ],Velocity.Swirling Vector Y [ s^-1 ],Velocity.Swirling Vector Z [ s^-1 ],Volume [ m^3 ],X [ m ],Y [ m ].1,Z [ m ].1
0,0.000028,-4.000000e-07,5.500000e-09,0.0,5.000000e-13,0.0001,0.0998,4.100000e-07,0.000200,0.0001,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000028,-4.000000e-07,5.500000e-09
1,0.000028,-3.800000e-07,5.500000e-09,0.0,1.000000e-12,0.0002,0.0998,4.250000e-07,0.000200,0.0001,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000028,-3.800000e-07,5.500000e-09
2,0.000028,-3.800000e-07,5.500000e-09,0.0,2.000000e-12,0.0004,0.0998,4.050000e-07,0.000200,0.0001,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000028,-3.800000e-07,5.500000e-09
3,0.000028,-4.000000e-07,5.500000e-09,0.0,1.000000e-12,0.0002,0.0998,4.060000e-07,0.000200,0.0001,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000028,-4.000000e-07,5.500000e-09
4,0.000028,-3.600000e-07,5.500000e-09,0.0,1.000000e-12,0.0002,0.0998,3.910000e-07,0.000200,0.0001,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000028,-3.600000e-07,5.500000e-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123036,-0.000002,3.130000e-07,5.500000e-09,0.0,1.080000e-12,0.0002,0.0998,3.870000e-07,0.000216,0.0001,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.000002,3.130000e-07,5.500000e-09
123037,-0.000002,3.350000e-07,5.500000e-09,0.0,1.080000e-12,0.0002,0.0998,3.630000e-07,0.000217,0.0001,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.000002,3.350000e-07,5.500000e-09
123038,-0.000002,3.560000e-07,5.500000e-09,0.0,1.090000e-12,0.0002,0.0998,3.330000e-07,0.000218,0.0001,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.000002,3.560000e-07,5.500000e-09
123039,-0.000002,3.780000e-07,5.500000e-09,0.0,1.090000e-12,0.0002,0.0998,2.930000e-07,0.000219,0.0001,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.000002,3.780000e-07,5.500000e-09


In [13]:
xtrain=df.iloc[:,0:55]
ytrain=df.iloc[:,55:]

In [14]:
xtrain.shape

(123041, 55)

In [15]:
ytrain.shape

(123041, 24)

In [16]:
inputs=keras.Input(shape=(55,))
x1=keras.layers.Dense(6,activation=keras.activations.selu)(inputs)
x2=keras.layers.Dense(12,activation=keras.activations.selu)(x1)
x3=keras.layers.Dense(24,activation=keras.activations.selu)(x2)
x4=keras.layers.Dense(48,activation=keras.activations.selu)(x3)
x5=keras.layers.Dense(96,activation=keras.activations.selu)(x4)
x6=keras.layers.Dense(192,activation=keras.activations.selu)(x5)
x7=keras.layers.Dense(200,activation=keras.activations.selu)(x6)
x8=keras.layers.Dense(200,activation=keras.activations.selu)(x7)
x9=keras.layers.Dense(224,activation=keras.activations.selu)(x8)
outputs=keras.layers.Dense(24)(x9)
model=keras.Model(inputs,outputs)
optimizer=keras.optimizers.Adam(learning_rate=1e-3)
model.summary()
batchsize=1
a=tf.convert_to_tensor(xtrain,dtype='float32').numpy()
b=tf.convert_to_tensor(ytrain,dtype='float32').numpy()
traindataset=tf.data.Dataset.from_tensor_slices((a,b))
#traindataset=traindataset.shuffle(buffer_size=100).batch(batchsize)
traindataset=traindataset.batch(batchsize)
def my_loss_fn(y_true,y_pred,x_true):
    squared_difference=tf.sqrt((y_true-y_pred)**2)
    return tf.reduce_mean(squared_difference)
epochs=1
new_p=[]
new_v=[]
listlossvalue=[]
liststep=[]
for epoch in range(epochs):
    print("\nStart of epoch %d"%(epoch,))
    for step,(xx,yy) in enumerate(traindataset):
        with tf.GradientTape(persistent=True) as tape:
            logits=model(xx,training=True)
            lossvalue=my_loss_fn(yy,logits,xx)
            listlossvalue.append(lossvalue)
            liststep.append(step)
            #new_p.append(logits.numpy()[0][0])
            #new_v.append(logits.numpy()[0][1])
        print(step+1)
        print('loss',lossvalue)
        grads=tape.gradient(lossvalue,model.trainable_variables)
        optimizer.apply_gradients(zip(grads,model.trainable_variables))
        
print('Thank you for your patience')  

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 55)]              0         
                                                                 
 dense_10 (Dense)            (None, 6)                 336       
                                                                 
 dense_11 (Dense)            (None, 12)                84        
                                                                 
 dense_12 (Dense)            (None, 24)                312       
                                                                 
 dense_13 (Dense)            (None, 48)                1200      
                                                                 
 dense_14 (Dense)            (None, 96)                4704      
                                                                 
 dense_15 (Dense)            (None, 192)               1862

117
loss tf.Tensor(0.011365163, shape=(), dtype=float32)
118
loss tf.Tensor(0.0112821115, shape=(), dtype=float32)
119
loss tf.Tensor(0.008578539, shape=(), dtype=float32)
120
loss tf.Tensor(0.008405431, shape=(), dtype=float32)
121
loss tf.Tensor(0.009030665, shape=(), dtype=float32)
122
loss tf.Tensor(0.01477664, shape=(), dtype=float32)
123
loss tf.Tensor(0.01792017, shape=(), dtype=float32)
124
loss tf.Tensor(0.042944778, shape=(), dtype=float32)
125
loss tf.Tensor(0.034728657, shape=(), dtype=float32)
126
loss tf.Tensor(0.03341491, shape=(), dtype=float32)
127
loss tf.Tensor(0.07899831, shape=(), dtype=float32)
128
loss tf.Tensor(0.11481258, shape=(), dtype=float32)


KeyboardInterrupt: 

In [23]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 55)]              0         
                                                                 
 dense_10 (Dense)            (None, 6)                 336       
                                                                 
 dense_11 (Dense)            (None, 12)                84        
                                                                 
 dense_12 (Dense)            (None, 24)                312       
                                                                 
 dense_13 (Dense)            (None, 48)                1200      
                                                                 
 dense_14 (Dense)            (None, 96)                4704      
                                                                 
 dense_15 (Dense)            (None, 192)               1862

In [ ]:
keras.utils.plot_model(model,"mids.png",show_shapes=['d'])

In [15]:
keras.callbacks.TensorBoard(log_dir="C:/Users/srbeh/Desktop/desktop docs/Final Year Project/8th sem/mids/hello")